In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf
from datetime import datetime

In [5]:
#def convert_to_date(x):
    return datetime.strptime(x, '%Y %m %d')

In [6]:
#data=pd.read_csv('twins_1992-1997.csv',parse_dates=[['year','month','day']],date_parser=convert_to_date,keep_date_col=False)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15728\3957504781.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data=pd.read_csv('twins_1992-1997.csv',parse_dates=[['year','month','day']],date_parser=convert_to_date,keep_date_col=False)


In [18]:
data=pd.read_csv('twins_1992-1997.csv')

In [20]:
data.head()

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.
0,1992,1,1,5.01,-110.00,-3.3,3.6,NaN,26.39,27.40
1,1992,1,1,1.98,-140.40,-4.4,4.1,NaN,27.75,28.27
2,1992,1,1,5.00,-140.00,NaN,NaN,NaN,26.70,28.27
3,1992,1,1,-2.00,156.00,9.6,4.3,NaN,27.77,29.39
4,1992,1,1,8.04,-110.06,NaN,NaN,86.3,26.72,27.30


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77506 entries, 0 to 77505
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   year       77506 non-null  int64  
 1   month      77506 non-null  int64  
 2   day        77506 non-null  int64  
 3   latitude   77506 non-null  float64
 4   longitude  77506 non-null  float64
 5   zon.winds  66895 non-null  float64
 6   mer.winds  66895 non-null  float64
 7   humidity   62674 non-null  float64
 8   air temp.  72158 non-null  float64
 9   s.s.temp.  77506 non-null  float64
dtypes: float64(7), int64(3)
memory usage: 5.9 MB


In [24]:
data['humidity'].interpolate(inplace=True)


In [26]:
data['zon.winds'].fillna(np.random.normal(data['zon.winds'].mean(),data['zon.winds'].std()),inplace=True)
data['mer.winds'].fillna(np.random.normal(data['mer.winds'].mean(),data['mer.winds'].std()),inplace=True)

In [28]:
data['humidity']=data['humidity'].fillna(data['humidity'].mean(),)

In [30]:
data=data.dropna(subset=['air temp.'])

In [32]:
data.count()

year         72158
month        72158
day          72158
latitude     72158
longitude    72158
zon.winds    72158
mer.winds    72158
humidity     72158
air temp.    72158
s.s.temp.    72158
dtype: int64

In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit


features = ['year', 'month', 'day', 'latitude', 'longitude', 'zon.winds', 'mer.winds', 'humidity', 'air temp.']
target = 's.s.temp.'


X = data[features]
y = data[target]
scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Fit the scaler on the training data and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=100,      # You can try reducing this number
    max_depth=10,          # Limit the depth of the trees
    min_samples_split=10,  # Increase the minimum samples required to split a node
    min_samples_leaf=4,    # Increase the minimum samples required at a leaf node
    random_state=42
)

# Train the model on the scaled training data
rf_model.fit(X_train_scaled, y_train)

# Predict on the scaled testing data
y_pred = rf_model.predict(X_test_scaled)

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 0.1464958190376019
R-squared: 0.9646981151809849


In [74]:
y_test_pred = rf_model.predict(X_test_scaled)
y_train_pred = rf_model.predict(X_train_scaled)

In [75]:
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

In [76]:
print("Training Data Performance")
print(f"Mean Squared Error: {mse_train}")
print(f"R-squared: {r2_train}")

Training Data Performance
Mean Squared Error: 0.12583908067889843
R-squared: 0.9693099356815261


In [77]:
print("\nTesting Data Performance")
print(f"Mean Squared Error: {mse_test}")
print(f"R-squared: {r2_test}")


Testing Data Performance
Mean Squared Error: 0.1464958190376019
R-squared: 0.9646981151809849
